In [44]:
import sync_wandb_run
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import utils as pu
from plot import create_plot_with_values
from plot_pretty import create_ultimate_plotly_plot, generate_scatter_plot, generate_scatter_plot_accuracy, generate_scatter_plot_gridscore
import plotly_utils

In [45]:
entity = "raphaelmaser"  # Your W&B username or team
project = "Improved-Interpretability-and-Concepts"  # Your W&B project name
df = sync_wandb_run.load_wandb_runs()
df.count()  # Display the number of runs loaded
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

val_accuracy = "Validation Accuracy"
grid_score = "PG Score"
frozen_layers = "config_define_frozen_layers"
trainable_params = "Trainable Parameters (M)"
time="Runtime (h)"
figsize=(10,4)


# All performed with ReLU and Kaiming

In [46]:
vit_c = df[df['group'].isin(["vit-conv-stem-lora", "vit-conv-stem"])] 
vit_c = vit_c[vit_c['config_lora_depth'] == 0]  # Base model
vit_c = vit_c[~vit_c["id"].isin(["znt9yrsg"])]  # Filter for rank 2
vit_c = vit_c[vit_c['name'] != "vit_b_16_B=2_pretrained_conv-stem-small"]  # Filter for the specific base model ID

vit_base = df[df['id'] == "2fupnc3h"] # Base model

lora = pd.concat([vit_c, vit_base], ignore_index=True)  # Add the Bcosified model to the DataFrame
# conv_wo_batchnorm = conv_wo_batchnorm[conv_wo_batchnorm["id"].isin(["sbpfkwct", "dv8y7o8k", "yheexwdv", "zf9oo2l6", "xka2n8rp"])]  # Filter for specific runs
# conv_wo_batchnorm = rename(conv_wo_batchnorm)  # Rename model names for better readability
lora.reset_index(drop=True, inplace=True)  # Reset index after filtering
print(f"Number of runs in layerwise_freezing group: {len(lora)}")

#Renaming
lora = pu.rename(lora).sort_values(by=val_accuracy, ascending=False)
lora = pu.rename_adapter_factor_experiments(lora)  # Rename columns for adapter factor experiments
lora.rename(columns={
    "config_lora_rank": "Lora Rank",
    "config_trainable_parameters": trainable_params,
    "summary__runtime": time,
}, inplace=True)

lora.replace({
    "Model": {
        "vitc_b_16_B=2_pretrained_lora_relu_kaiming": "ViTc LoRA (ReLU, Kaiming)",
        "vit_b_16_B=2_pretrained_conv-stem": "ViTc Bcosified",
        "vitc_b_16_B=2_pretrained": "ViTc Bcosified",
        "vitc_b_16_B=2_pretrained_only-conv-stem": "Only Conv Stem",
        "vit_b_16_B=2_pretrained": "ViT Bcosified",
    }}, inplace=True)  # Rename specific model names for clarity

lora[time] = lora[time].apply(lambda x: x/(60*60))  # Format runtime to 2 decimal places
# Some stats
for name in lora["Model"].unique(): print(name)  # Print each model name
display(lora)


Number of runs in layerwise_freezing group: 15
ViTc Bcosified
ViTc LoRA (ReLU, Kaiming)
Only Conv Stem
ViT Bcosified


id                      Model     state        entity  \
9   n8dqbqan             ViTc Bcosified  finished  raphaelmaser   
5   1zn8qgax  ViTc LoRA (ReLU, Kaiming)  finished  raphaelmaser   
10  ujensn29  ViTc LoRA (ReLU, Kaiming)  finished  raphaelmaser   
12  sdfqpo0q  ViTc LoRA (ReLU, Kaiming)  finished  raphaelmaser   
1   ck8pz3ho  ViTc LoRA (ReLU, Kaiming)  finished  raphaelmaser   
4   06xy8vza  ViTc LoRA (ReLU, Kaiming)  finished  raphaelmaser   
2   cs0o0f2r  ViTc LoRA (ReLU, Kaiming)  finished  raphaelmaser   
11  8vc4fqpn  ViTc LoRA (ReLU, Kaiming)  finished  raphaelmaser   
3   estm6rl0  ViTc LoRA (ReLU, Kaiming)  finished  raphaelmaser   
7   l71baxrr  ViTc LoRA (ReLU, Kaiming)  finished  raphaelmaser   
0   5kw5297l  ViTc LoRA (ReLU, Kaiming)  finished  raphaelmaser   
6   akb0m889             ViTc Bcosified  finished  raphaelmaser   
13  uaryeifu             Only Conv Stem  finished  raphaelmaser   
14  2fupnc3h              ViT Bcosified  finished  raphaelmaser   
8   3rxn5wyt  ViTc LoRA (ReLU, Kaiming)  finished  raphaelmaser   

                                   project            created_at  updated_at  \
9   Improved-Interpretability-and-Concepts  2025-07-19T06:49:42Z         NaN   
5   Improved-Interpretability-and-Concepts  2025-07-19T06:56:45Z         NaN   
10  Improved-Interpretability-and-Concepts  2025-07-20T08:46:53Z         NaN   
12  Improved-Interpretability-and-Concepts  2025-07-20T08:46:29Z         NaN   
1   Improved-Interpretability-and-Concepts  2025-07-19T21:35:35Z         NaN   
4   Improved-Interpretability-and-Concepts  2025-07-19T22:40:52Z         NaN   
2   Improved-Interpretability-and-Concepts  2025-07-22T15:11:15Z         NaN   
11  Improved-Interpretability-and-Concepts  2025-07-22T15:17:49Z         NaN   
3   Improved-Interpretability-and-Concepts  2025-07-22T15:11:15Z         NaN   
7   Improved-Interpretability-and-Concepts  2025-07-22T15:08:23Z         NaN   
0   Improved-Interpretability-and-Concepts  2025-07-22T15:20:57Z         NaN   
6   Improved-Interpretability-and-Concepts  2025-07-22T19:21:40Z         NaN   
13  Improved-Interpretability-and-Concepts  2025-07-21T10:28:29Z         NaN   
14  Improved-Interpretability-and-Concepts  2025-07-19T06:48:52Z         NaN   
8   Improved-Interpretability-and-Concepts  2025-07-22T15:20:52Z         NaN   

   tags notes                                                url  \
9   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
5   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
10  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
12  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
1   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
4   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
2   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
11  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
3   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
7   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
0   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
6   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
13  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
14  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
8   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   

                 group  job_type  sweep                   synced_at  config_b  \
9        vit-conv-stem       NaN    NaN  2025-07-27T10:49:36.411784       2.0   
5   vit-conv-stem-lora       NaN    NaN  2025-07-27T10:49:38.005564       2.0   
10  vit-conv-stem-lora       NaN    NaN  2025-07-27T10:49:41.667236       2.0   
12  vit-conv-stem-lora       NaN    NaN  2025-07-27T10:49:40.758101       2.0   
1   vit-conv-stem-lora       NaN    NaN  2025-07-27T10:49:38.970541       2.0   
4   vit-conv-stem-lora       NaN    NaN  2025-07-27T10:49:39.906745       2.0   


In [47]:
for i in lora.notna().keys(): print(i)

id
Model
state
entity
project
created_at
updated_at
tags
notes
url
group
job_type
sweep
synced_at
config_b
config_lr
config_loss
config_seed
config_tags
config_group
config_model
config_epochs
config_adapter
config_compile
config_dataset
config_vit_dim
config_use_lora
config_act_layer
config_b_adapter
config_data_path
config_precision
config_vit_depth
config_vit_qkv_b
config_batch_size
config_cache_data
config_experiment
config_image_size
config_norm_layer
config_pin_memory
config_block_class
config_convolution
config_lora_factor
config_num_classes
config_num_workers
config_vit_mlp_dim
config_b_classifier
config_compile_mode
config_dynamic_bias
config_layergradcam
config_linear_layer
config_weight_decay
config_grid_pg_scale
config_vit_conv_stem
config_vit_num_heads
config_warmup_epochs
config_conv_adapter_b
config_dynamic_bias_b
config_efficient_bcos
config_vit_patch_size
config_concept_layer_b
config_experiment_name
config_matmul_precision
config_compile_fullgraph
config_concept_layer

In [48]:
# vit_base_wd_0= lora[lora['config_weight_decay'] == 0]  # Filter for weight decay 0.0

# plt.figure(figsize=figsize)
# ax = sns.scatterplot(
#     data=vit_base_wd_0,
#     y=val_accuracy,
#     x=trainable_params,
#     hue="Model",  # Color points by name
#     # s=100  # Make points larger
# )
# #pu.add_val_to_plot(ax)
# plt.ylim(0.7, 0.9)
# # Move legend outside the plot
# plt.tight_layout()
# plt.show()


# plt.figure(figsize=figsize)
# ax = sns.scatterplot(
#     data=vit_base_wd_0,
#     y=grid_score,
#     x=trainable_params,
#     hue="Model",  # Color points by name
#     # s=100  # Make points larger
# )
# #pu.add_val_to_plot(ax)
# plt.ylim(0.7, 1)
# # Move legend outside the plot
# plt.tight_layout()
# plt.show()

In [49]:
vit_base_wd_0= lora[lora['config_weight_decay'] == 0]  # Filter for weight decay 0.0

# create_plot_with_values(
#     data=vit_base_wd_0,
#     y=val_accuracy,
#     x=trainable_params,
#     hue="Model",  # Color points by name
#     figsize=(10, 4),
#     #value_position="below",
#     ylim=(0.2, 1),
#     show_scale_lines=True,
#     show_scale_values=True,
#     max_scale_points=10,
#     show_default_xticks=True
# )

# create_plot_with_values(
#     data=vit_base_wd_0,
#     y=grid_score,
#     x=trainable_params,
#     hue="Model",  # Color points by name
#     figsize=(10, 4),
#     #value_position="below",
#     ylim=(0.5, 1),
#     show_scale_lines=True,
#     show_scale_values=True,
#     max_scale_points=10,
#     show_default_xticks=True
# )

vit_base_wd_0= lora[lora['config_weight_decay'] == 0]  # Filter for weight decay 0.0

generate_scatter_plot_accuracy(
    data=vit_base_wd_0,
    x=trainable_params,
    y=val_accuracy,
    color="Model",
    scale_factor=2,
    title="ViTc - No Weight Decay",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)


In [50]:
generate_scatter_plot_gridscore(
    data=vit_base_wd_0,
    x=trainable_params,
    y=grid_score,
    color="Model",
    scale_factor=2,
    title="ViTc - No Weight Decay",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

## Weight decay = 0.1

In [51]:
vit_base_wd_01= lora[lora['config_weight_decay'] == 0.1]  # Filter for weight decay 0.0

generate_scatter_plot_accuracy(
    data=vit_base_wd_01,
    x=trainable_params,
    y=val_accuracy,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

In [52]:
generate_scatter_plot_gridscore(
    data=vit_base_wd_01,
    x=trainable_params,
    y=grid_score,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

In [54]:
vit_base_wd_01= lora[lora['config_weight_decay'] == 0.1]  # Filter for weight decay 0.0

generate_scatter_plot_accuracy(
    data=vit_base_wd_01,
    x=trainable_params,
    y=time,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params,
    range_y=(0, 7)
)